# Acquiring data (extraction)

<img src='images/flow-extract.png' width=800>

> Note: in some organizations, there is a data discovery system, like https://www.amundsen.io/amundsen/ upstream from this step. We're not covering that area due to scope constraints


## Goal: use SQL to efficiently retrieve data for further work

### Legacy Tools

Mostly: Apache Hive

### Current Tools

* SparkSQL
* Presto
* *Hive Metastore*

### Rising/Future Tools

* Kartothek, Intake
* BlazingSQL
* Dask-SQL

*There are more non-SQL options, but support for SQL is a requirement in most large organizations, so we're sticking with SQL-capable tools for now*


In [ ]:
import pyspark

In [ ]:
spark = pyspark.sql.SparkSession.builder.appName("demo").getOrCreate()

In [ ]:
spark.sql("SELECT * FROM parquet.`data/california`").show()

In [ ]:
query = """
SELECT origin, mean(delay) as delay, count(1) 
FROM parquet.`data/california` 
GROUP BY origin
HAVING count(1) > 500
ORDER BY delay DESC
"""
spark.sql(query).show()

In [ ]:
query = """
SELECT *
FROM parquet.`data/california` 
WHERE origin in (
    SELECT origin 
    FROM parquet.`data/california` 
    GROUP BY origin 
    HAVING count(1) > 500
)
"""
spark.sql(query).write.mode('overwrite').option('header', 'true').csv('data/refined_flights/')

In [ ]:
! head data/refined_flights/*.csv